### Obtain List of Categorical/Oridnal Data Column Names from Excel Spread Sheet

In [2]:
import pandas as pd
import numpy as np

In [3]:
#read train data and test data
data_train=pd.read_csv('train.csv')
testData=pd.read_csv('test.csv')

In [4]:
data_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Imputing Missing Data

In [5]:
# Create Function---Change missing values to 'NA' for list of columns 
def fill_NA_missing_value (dataFrame, list_of_ColumnName):
    dataFrame[list_of_ColumnName]=dataFrame[list_of_ColumnName].fillna(value='None')
    return dataFrame

# Create Function---Change missing values to '0' for list of columns 
def fill_0_missing_value (dataFrame, list_of_ColumnName):
    dataFrame[list_of_ColumnName]=dataFrame[list_of_ColumnName].fillna(value=0)
    return dataFrame

In [6]:
#print columns with missing value (nan) from train data
data_train.columns[data_train.isnull().any()]

Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence',
       'MiscFeature'],
      dtype='object')

In [7]:
#print columns with missing values from test data
testData.columns[testData.isnull().sum()>0]

Index(['MSZoning', 'LotFrontage', 'Alley', 'Utilities', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath',
       'BsmtHalfBath', 'KitchenQual', 'Functional', 'FireplaceQu',
       'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea',
       'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType'],
      dtype='object')

In [8]:
def data_processing(dataFrame):
    data_process=dataFrame.copy()
    
    #ID 
    data_process.set_index('Id') 
    '''mia df_raw = pd.read_csv('train.csv',index_col=0) '''


    #MSSubclass-Categorical-fill missing value to NONE--change dataType to category 
    data_process['MSSubClass'] = data_process['MSSubClass'].fillna("None")
    data_process['MSSubClass'] = data_process['MSSubClass'].astype('category')
    
    #MSZoning-Categorical-fill missing value to mode-- most frequent category is RL so we use mode--change dataType to category
    data_process['MSZoning']=data_process['MSZoning'].fillna(data_process.MSZoning.mode()[0])
    data_process['MSZoning'] = data_process['MSZoning'].astype('category')

    #LotFrontage-numerical-since houses in similar neighborhood probably have similar area, so we use median based on the neighborhood 
    data_process.LotFrontage = data_process.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))

    #LotArea--numerical
    data_process['LotArea']=data_process.LotArea.fillna(0)

    #Alley--categorical NA means no alley access
    data_process.Alley=data_process.Alley.fillna('AlleyNA')

    #MasVnrType: NA means no MasVnr so fill with NA and 0 for the area
    data_process["MasVnrType"] = data_process["MasVnrType"].fillna("MasNone")
    data_process["MasVnrArea"] = data_process["MasVnrArea"].fillna(0)
     
    
    #Utilities: there are 2 NaN in the test dataset. 
    #data_process.drop(['Utilities'], axis=1,inplace=True)

    #Utilities: there are 2 NaN in the test dataset. 
    
    #Basement related variables: 'BsmtQual','BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'
    #Nan means no basement
    data_process=fill_NA_missing_value(data_process, ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'])

    #Electrical: There is only 1 missing value for electrical. We will use mode 
    data_process['Electrical']=data_process['Electrical'].fillna(data_process.Electrical.mode()[0])

    #FireplaceQu: missing value means no fireplace
    data_process["FireplaceQu"] = data_process["FireplaceQu"].fillna("FireplaceNone")

    #Garage related categorical variables: 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond'
    #Missing means no garage
    data_process['GarageType'] = data_process["GarageType"].fillna("GarageNone")
    data_process=fill_NA_missing_value(data_process, ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'])


    #Garage related numerical variables:  GarageYrBlt, GarageCars, GarageArea, 
    #Missing means no garage
    data_proces=fill_0_missing_value(data_process, ["GarageYrBlt", "GarageCars", "GarageArea"] )

    #PoolQC: missing means no pool
    data_process['PoolQC'] = data_process["PoolQC"].fillna("PoolNone")

    #Fence: na means no Fence
    data_process['Fence'] = data_process["Fence"].fillna("FenceNA")

    #MiscFeature: na means no MiscFeature
    data_process['MiscFeature'] = data_process["MiscFeature"].fillna("MiscFeature")

    #Exterior1st, Exterior2nd both have missing values in the test dataset and 0 in the training dataset. 
    #set the missing value with the mode
    data_process['Exterior1st']=data_process['Exterior1st'].fillna(data_process.Exterior1st.mode()[0])
    data_process['Exterior2nd']=data_process['Exterior1st'].fillna(data_process.Exterior1st.mode()[0])

    #TotalBsmtSF is the sum of BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, so we drop TotalBsmtSF
    #data_process.drop(['TotalBsmtSF'], axis=1,inplace=True)

    #BsmtFullBath, BsmtHalfBath, BsmtFinSF1, BsmtFinSF2, BsmtUnfSF: if there is na, it means there is no basement
    data_process=fill_0_missing_value(data_process, ["BsmtFullBath", "BsmtHalfBath" , "BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF"])

    #KitchenQual, there is 1 missing value at the test set. set missing value with the most frequently appeared category
    data_process['KitchenQual'] = data_process['KitchenQual'].fillna(data_process['KitchenQual'].mode()[0])

    #Functional, there are 2 missing value at the test set. set missing value with the most frequently appeared category
    data_process['Functional'] = data_process['Functional'].fillna(data_process['Functional'].mode()[0])

    #SaleType, there is 1 missing value at the test set. set missing value with the most frequently appeared category
    data_process['SaleType'] = data_process['SaleType'].fillna(data_process['SaleType'].mode()[0])


    #PROCESS DATA THAT ARE NOT MISSING IN THE TRAINING AND TEST
    
    return data_process

In [9]:
#Process trian and test data
data_train=data_processing(data_train)
testData=data_processing(testData)

In [10]:
data_train.isnull().sum().sum()

0

In [11]:
testData.isnull().sum().sum()

3

In [12]:
###READ EXCEL____NEED TO BE REMOVED AFTER CLEANING 
val=pd.read_excel('Variable_Analysis_11.07.xlsx')
val.head()
val.Name[val.Team_dataType=='numerical'].tolist()

['LotFrontage',
 'LotArea',
 'YearBuilt',
 'YearRemodAdd',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'SalePrice']

In [13]:
List_colName_Categorical=['MSSubClass',
 'MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'Foundation',
 'Heating',
 'CentralAir',
 'Electrical',
 'GarageType',
 'PavedDrive',
 'MiscFeature']

len(List_colName_Categorical)

25

In [14]:
List_Column_ordinal=['OverallQual',
 'OverallCond',
 'ExterQual',
 'ExterCond',
 'BsmtQual',
 'BsmtCond',
 'BsmtFinType1',
 'BsmtFinType2',
 'HeatingQC',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence']

len(List_Column_ordinal)

17

In [15]:
List_of_Column_Numerical=['LotFrontage',
 'LotArea',
 'YearBuilt',
 'YearRemodAdd',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'SalePrice']
len(List_of_Column_Numerical)

31

In [16]:
30+17+25+7

79

# Feature Engineering

### Dummify for Categorical Data

In [17]:
def Process_Categorical_Data (dataFrame, List_colName_Categorical):
    data=dataFrame.copy()
    for column_name in List_colName_Categorical:
        dummied_data=pd.get_dummies(data[column_name], prefix=column_name,prefix_sep='__')
        dummied_data=dummied_data.drop(dummied_data.columns[0], axis=1)#drop 1st column
        data=pd.concat([data.drop(column_name,axis=1), dummied_data ], axis=1)
    return data
        

In [18]:
#Review columns after Dummification
data_train=Process_Categorical_Data(data_train, List_colName_Categorical)
list(data_train.columns.values)

['Id',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'HeatingQC',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'PoolQC',
 'Fence',
 'MiscVal',
 'MoSold',
 'YrSold',
 'SaleType',
 'SaleCondition',
 'SalePrice',
 'MSSubClass__30',
 'MSSubClass__40',
 'MSSubClass__45',
 'MSSubClass__50',
 'MSSubClass__60',
 'MSSubClass__70',
 'MSSubClass__75',
 'MSSubClass__80',
 'MSSubClass__85',
 'MSSub

### Process Ordinal Data

In [19]:

data_train.replace({
        "BsmtCond" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
        "ExterQual" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
        "ExterCond" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
        "BsmtQual" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5},
         "BsmtFinType1" : {"None" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4,
                                         "ALQ" : 5, "GLQ" : 6},
        "BsmtFinType2" : {"None" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4,
                                         "ALQ" : 5, "GLQ" : 6},
        'HeatingQC':{"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
        'KitchenQual':{"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
        'Functional':{"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
        'FireplaceQu':{"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
         "GarageCond" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
        "GarageQual" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
        "PoolQC" : {"None" : 0, "Fa" : 1, "TA" : 2, "Gd" : 3, "Ex" : 4},
        "GarageFinish": {"None":0, 'Unf':1, 'RFn': 2, 'Fin':3 }     
})
    
 #'OverallQual',: use binning or as numerical?
 #'OverallCond': use binning or as numerical?
#BsmtUnfSF: numerical??
#why is fence ordinal????????????

# ['OverallQual',????
#  'OverallCond',????
#  'ExterQual',
#  'ExterCond',
#  'BsmtQual',
#  'BsmtCond',
#  'BsmtFinType1',
#  'BsmtFinType2',
#  'HeatingQC',
#  'KitchenQual',
#  'Functional',
#  'FireplaceQu',
#  'GarageFinish',
#  'GarageQual',
#  'GarageCond',
#  'PoolQC',
#  'Fence']





,Id,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrType,MasVnrArea,ExterQual,...,GarageType__BuiltIn,GarageType__CarPort,GarageType__Detchd,GarageType__GarageNone,PavedDrive__P,PavedDrive__Y,MiscFeature__MiscFeature,MiscFeature__Othr,MiscFeature__Shed,MiscFeature__TenC
0,1,65.0,8450,7,5,2003,2003,BrkFace,196.0,4,...,0,0,0,0,0,1,1,0,0,0
1,2,80.0,9600,6,8,1976,1976,None,0.0,3,...,0,0,0,0,0,1,1,0,0,0
2,3,68.0,11250,7,5,2001,2002,BrkFace,162.0,4,...,0,0,0,0,0,1,1,0,0,0
3,4,60.0,9550,7,5,1915,1970,None,0.0,3,...,0,0,1,0,0,1,1,0,0,0
4,5,84.0,14260,8,5,2000,2000,BrkFace,350.0,4,...,0,0,0,0,0,1,1,0,0,0
5,6,85.0,14115,5,5,1993,1995,None,0.0,3,...,0,0,0,0,0,1,0,0,1,0
6,7,75.0,10084,8,5,2004,2005,Stone,186.0,4,...,0,0,0,0,0,1,1,0,0,0
7,8,80.0,10382,7,6,1973,1973,Stone,240.0,3,...,0,0,0,0,0,1,0,0,1,0
8,9,51.0,6120,7,5,1931,1950,None,0.0,3,...,0,0,1,0,0,1,1,0,0,0
9,10,50.0,7420,5,6,1939,1950,None,0.0,3,...,0,0,0,0,0,1,1,0,0,0


In [59]:
# def ordinal_processing_None_Ex (dataFrame, List_colName):
#     data=dataFrame.copy()
#     scale_mapper={
#         'None': 0,
#         'Po':1, 
#         'Fa':2,         
#         'TA':3,
#         'Gd':4,    
#         'Ex':5 }
#     for column_name in List_colName:
#         if data[column_name].dtype in ['int32', 'int64']:
#             continue
#         else:
#             data[column_name]=data[column_name].replace(scale_mapper)
#     return data
            

In [69]:
testData.GarageFinish.unique()

array(['Unf', 'Fin', 'RFn', 'None'], dtype=object)

In [96]:
data_train.HeatingQC.unique()

array(['Ex', 'Gd', 'TA', 'Fa', 'Po'], dtype=object)

### Check the numerical data dtypes

## Fit Data to the Multi-Linear Regression Model

In [20]:
X_train=data_train.drop(columns='SalePrice')
X_train.head()

,Id,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrType,MasVnrArea,ExterQual,...,GarageType__BuiltIn,GarageType__CarPort,GarageType__Detchd,GarageType__GarageNone,PavedDrive__P,PavedDrive__Y,MiscFeature__MiscFeature,MiscFeature__Othr,MiscFeature__Shed,MiscFeature__TenC
0,1,65.0,8450,7,5,2003,2003,BrkFace,196.0,Gd,...,0,0,0,0,0,1,1,0,0,0
1,2,80.0,9600,6,8,1976,1976,None,0.0,TA,...,0,0,0,0,0,1,1,0,0,0
2,3,68.0,11250,7,5,2001,2002,BrkFace,162.0,Gd,...,0,0,0,0,0,1,1,0,0,0
3,4,60.0,9550,7,5,1915,1970,None,0.0,TA,...,0,0,1,0,0,1,1,0,0,0
4,5,84.0,14260,8,5,2000,2000,BrkFace,350.0,Gd,...,0,0,0,0,0,1,1,0,0,0


In [21]:
y_train=data_train['SalePrice']
y_train_log=data_train['SalePrice'].apply(lambda x: np.log(x))

In [22]:
from sklearn.linear_model import LinearRegression 

ols = LinearRegression()
ols.fit(X_train, y_train_log)
print("R^2 for train set: %f" %ols.score(X_train, y_train_log))
print("Intercept: %f" %ols.intercept_)

ValueError: could not convert string to float: 'BrkFace'